In [1]:
import os, sys, pathlib
REPO = pathlib.Path.cwd()  # start Jupyter in the repo root
assert (REPO / "demo.py").exists(), "Run Jupyter from the repo folder"
sys.path.insert(0, str(REPO))   # let Python find 'detectors'
print("Repo:", REPO)

Repo: C:\Users\yonda\Downloads\OptIForest-main


In [2]:
import pandas as pd
src = REPO/"data"/"ad.csv"
dst = REPO/"data"/"ad_1k.csv"
df = pd.read_csv(src, header=None).sample(1000, random_state=0)
df.to_csv(dst, index=False, header=False)
len(df), dst

(1000, WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/data/ad_1k.csv'))

In [3]:
import subprocess, time

DATASET, THRESH, BRANCH = "ad_1k", "200", "3"   # quick settings
ART = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
log = ART / f"run_{DATASET}_thr{THRESH}_br{BRANCH}_{ts}.txt"

cmd = [sys.executable, "-u", str(REPO/"demo.py"),
       "--dataset", DATASET, "--threshold", THRESH, "--branch", BRANCH]
env = {**os.environ, "PYTHONPATH": str(REPO)}

print("COMMAND:", " ".join(cmd))
print("LOG    ->", log)

with open(log, "w", encoding="utf-8") as f:
    p = subprocess.Popen(cmd, cwd=str(REPO), env=env,
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        print(line, end=""); f.write(line)
    rc = p.wait()
print("Exit code:", rc)

COMMAND: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad_1k --threshold 200 --branch 3
LOG    -> C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_ad_1k_thr200_br3_20250825T063953Z.txt
Dataset: ad_1k

L2OPT:
	AUC score:	 0.7693101919472982
	AUC std:	 0.006128793950627537
	PR score:	 0.45043243315134623
	PR std:	 0.02949784324061669
	Training time:	 36.992628971735634
	Testing time:	 51.56210856437683
Exit code: 0


In [4]:
import subprocess
(ART / f"python_version_{ts}.txt").write_text(subprocess.check_output([sys.executable,"-V"], text=True))
(ART / f"requirements_{ts}.txt").write_text(subprocess.check_output([sys.executable,"-m","pip","freeze"], text=True))
try:
    commit = subprocess.check_output(["git","rev-parse","HEAD"], cwd=str(REPO), text=True).strip()
except Exception:
    commit = f"ZIP download at {ts}"
(ART / f"repo_version_{ts}.txt").write_text(commit+"\n")
list(ART.iterdir())

[WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/.ipynb_checkpoints'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/python_version.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/python_version_20250825T063953Z.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/repo_version.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/repo_version_20250825T063953Z.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/requirements.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/requirements_20250825T063953Z.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/run_ad_1k_thr200_br3_20250825T063953Z.txt')]

In [5]:
import os, sys, time, subprocess, pathlib, re, pandas as pd

# paths
REPO = pathlib.Path.cwd()              # start Jupyter from the repo root
DEMO = REPO / "demo.py"
ART  = REPO / "replication_artifacts"
ART.mkdir(exist_ok=True)

# experiment settings (adjust only if you want)
DATASET   = "ad_1k"     # your 1k subsample file in data/
THRESHOLD = "200"       # good for small subsets
BRANCHES  = [0, 2, 3, 4]

rows = []
for br in BRANCHES:
    ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts}.txt"
    cmd = [sys.executable, "-u", str(DEMO),
           "--dataset", DATASET, "--threshold", str(THRESHOLD), "--branch", str(br)]
    env = {**os.environ, "PYTHONPATH": str(REPO)}

    print("RUN:", " ".join(cmd))
    print("LOG ->", log)

    # run and capture live output + file log
    with open(log, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd, cwd=str(REPO), env=env,
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            print(line, end=""); f.write(line)
        p.wait()

    # parse the metrics we need from the log
    txt = log.read_text(encoding="utf-8", errors="ignore")
    def grab(pat):
        m = re.search(pat, txt)
        return float(m.group(1)) if m else float("nan")
    rows.append({
        "branch":  br,
        "auc_mean": grab(r"AUC score:\s*([0-9.]+)"),
        "auc_std":  grab(r"AUC std:\s*([0-9.]+)"),
        "pr_mean":  grab(r"PR score:\s*([0-9.]+)"),
        "pr_std":   grab(r"PR std:\s*([0-9.]+)"),
        "train_s":  grab(r"Training time:\s*([0-9.]+)"),
        "test_s":   grab(r"Testing time:\s*([0-9.]+)")
    })

df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
df

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad_1k --threshold 200 --branch 0
LOG -> C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_ad_1k_thr200_br0_20250825T071257Z.txt
Dataset: ad_1k

L2OPT:
	AUC score:	 0.7689168863031172
	AUC std:	 0.005190100248176101
	PR score:	 0.4576097484797882
	PR std:	 0.02182053072631033
	Training time:	 35.58975345293681
	Testing time:	 50.75890011787415
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad_1k --threshold 200 --branch 2
LOG -> C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_ad_1k_thr200_br2_20250825T073434Z.txt
Dataset: ad_1k

L2OPT:
	AUC score:	 0.7635400771923525
	AUC std:	 0.004449921375708635
	PR score:	 0.4129919095926011
	PR std:	 0.026269668381486973
	Training time:	 33.22372914950053
	Testing 

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.768917,0.005190,0.457610,0.021821,35.589753,50.758900
1,2,0.763540,0.004450,0.412992,0.026270,33.223729,53.350875
2,3,0.771447,0.006889,0.459655,0.030280,36.082078,49.653521
3,4,0.773292,0.004519,0.467464,0.020155,36.222123,49.411256


In [6]:
from io import StringIO

DATASET   = "ad_1k"
THRESHOLD = "200"

# 1) CSV
csv_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.csv"
df.to_csv(csv_path, index=False)

# 2) Markdown
md = StringIO()
md.write("| branch | ROC-AUC (mean±std) | PR-AUC (mean±std) | Train s | Test s |\n")
md.write("|:------:|:-------------------:|:-----------------:|:-------:|:------:|\n")
for _, r in df.iterrows():
    md.write(f"| {int(r.branch)} | {r.auc_mean:.4f}±{r.auc_std:.4f} | {r.pr_mean:.4f}±{r.pr_std:.4f} | {r.train_s:.2f} | {r.test_s:.2f} |\n")
md_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.md"
md_path.write_text(md.getvalue(), encoding="utf-8")

# 3) LaTeX (booktabs style)
latex = StringIO()
latex.write("\\begin{table}[h]\n\\centering\n\\begin{tabular}{rcccc}\n\\toprule\n")
latex.write("Branch & ROC-AUC (mean$\\pm$std) & PR-AUC (mean$\\pm$std) & Train (s) & Test (s) \\\\\n\\midrule\n")
for _, r in df.iterrows():
    latex.write(f"{int(r.branch)} & {r.auc_mean:.4f}$\\pm${r.auc_std:.4f} & {r.pr_mean:.4f}$\\pm${r.pr_std:.4f} & {r.train_s:.2f} & {r.test_s:.2f} \\\\\n")
latex.write("\\bottomrule\n\\end{tabular}\n\\caption{OptIForest ablation on \\texttt{%s} (threshold %s).}\n\\label{tab:optiforest-ablation}\n\\end{table}\n" % (DATASET, THRESHOLD))
tex_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.tex"
tex_path.write_text(latex.getvalue(), encoding="utf-8")

csv_path, md_path, tex_path

(WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_ad_1k_thr200.csv'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_ad_1k_thr200.md'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_ad_1k_thr200.tex'))

# Existing Datasets application

## Wisoconsin Breast Cancer

## Setup

In [1]:
import os, sys, time, pathlib

REPO = pathlib.Path.cwd()
DATA = REPO / "data"
ART  = REPO / "replication_artifacts"
ART.mkdir(exist_ok=True)

# Make sure Python can import the repo package (detectors/)
if str(REPO) not in sys.path:
    sys.path.insert(0, str(REPO))
os.environ["PYTHONPATH"] = str(REPO)

print("Repo:", REPO)
print("demo.py exists:", (REPO/"demo.py").exists())
print("detectors/ exists:", (REPO/"detectors").exists())

Repo: C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main
demo.py exists: True
detectors/ exists: True


## Conversion from UCI raw to CSV

In [3]:
import pandas as pd, numpy as np
from datetime import datetime

# Try UCI .data first
UCI_DATA = DATA/"breast-cancer-wisconsin.data"    # 699 rows typical; may contain '?'
RAW_CSV  = None  # e.g., DATA/"breastw_raw.csv" if you downloaded a CSV

if UCI_DATA.exists():
    cols = ["id","clump_thickness","cell_size_uniformity","cell_shape_uniformity","marginal_adhesion",
            "single_epithelial_cell_size","bare_nuclei","bland_chromatin","normal_nucleoli","mitoses","class"]
    df = pd.read_csv(UCI_DATA, header=None, names=cols, na_values=["?"])
    # Drop missing; drop ID
    before = len(df)
    df = df.dropna().drop(columns=["id"]).reset_index(drop=True)
    print(f"[convert] loaded UCI .data: {before} rows -> {len(df)} rows after dropping NA and ID")
elif RAW_CSV and pathlib.Path(RAW_CSV).exists():
    df = pd.read_csv(RAW_CSV)
    # Ensure columns contain 'class' and no non-numeric feature cols
    assert "class" in df.columns, "Expected a 'class' column in your CSV"
    # If an ID-like column exists, drop it (name varies)
    for cand in ["id", "ID", "Id", "sample_code_number"]:
        if cand in df.columns:
            df = df.drop(columns=[cand])
else:
    raise FileNotFoundError("Place the UCI file at data/breast-cancer-wisconsin.data or set RAW_CSV to your CSV path.")

# Map labels: malignant(4)->1 (anomaly), benign(2)->0
y = (df["class"].astype(int) == 4).astype(int)
X = df.drop(columns=["class"])

# Save in repo format: features..., label_last, no header
OUT = DATA/"breastw.csv"
pd.DataFrame(np.c_[X.to_numpy(), y.to_numpy()]).to_csv(OUT, index=False, header=False)
print(f"[convert] wrote {OUT} with shape {X.shape[0]} x {X.shape[1]+1}")

# Print & save dataset stats (good for your appendix)
rows, feats = X.shape
pos_rate = float(y.mean())
stats_txt = (
    f"dataset=breastw\nrows={rows}\nfeatures={feats}\npositive_rate(malignant_as_anomaly)={pos_rate:.4f}\n"
)
print(stats_txt)

ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
(ART / f"dataset_breastw_stats_{ts}.txt").write_text(stats_txt, encoding="utf-8")

[convert] loaded UCI .data: 699 rows -> 683 rows after dropping NA and ID
[convert] wrote C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\data\breastw.csv with shape 683 x 10
dataset=breastw
rows=683
features=9
positive_rate(malignant_as_anomaly)=0.3499



79

## Running OptlForest considering branch 3

In [4]:
import subprocess

DATASET   = "breastw"
THRESHOLD = "200"
BRANCH    = "3"   # you can ablate later

ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{BRANCH}_{ts}.txt"
cmd = [sys.executable, "-u", str(REPO/"demo.py"),
       "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", BRANCH]

print("COMMAND:", " ".join(cmd))
print("LOG    ->", log)

with open(log, "w", encoding="utf-8") as f:
    p = subprocess.Popen(cmd, cwd=str(REPO),
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        print(line, end=""); f.write(line)
    rc = p.wait()
print("Exit code:", rc)
print("Log saved:", log)

COMMAND: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw --threshold 200 --branch 3
LOG    -> C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_breastw_thr200_br3_20250826T131846Z.txt
Dataset: breastw

L2OPT:
	AUC score:	 0.9880771985374497
	AUC std:	 0.0012793239778485662
	PR score:	 0.9745911601538693
	PR std:	 0.0035231599110506885
	Training time:	 27.350492493311563
	Testing time:	 23.946810499827066
Exit code: 0
Log saved: C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_breastw_thr200_br3_20250826T131846Z.txt


## Saving environment details for appendix

In [5]:
import subprocess

# Python version
(py_ver := ART / f"python_version_{ts}.txt").write_text(
    subprocess.check_output([sys.executable, "-V"], text=True), encoding="utf-8"
)

# Pip freeze
(reqs := ART / f"requirements_{ts}.txt").write_text(
    subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True), encoding="utf-8"
)

# Repo version (git commit or ZIP timestamp)
try:
    commit = subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=str(REPO), text=True).strip()
except Exception:
    commit = f"ZIP download at {ts}"
(rv := ART / f"repo_version_{ts}.txt").write_text(commit + "\n", encoding="utf-8")

py_ver, reqs, rv

(WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/python_version_20250826T131846Z.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/requirements_20250826T131846Z.txt'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/repo_version_20250826T131846Z.txt'))

## Ablation loop

In [6]:
import re, pandas as pd
BRANCHES = [0,2,3,4]
rows = []

for br in BRANCHES:
    ts2  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log2 = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts2}.txt"
    cmd2 = [sys.executable, "-u", str(REPO/"demo.py"),
            "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", str(br)]
    print("RUN:", " ".join(cmd2))
    with open(log2, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd2, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            print(line, end=""); f.write(line)
        p.wait()
    txt = log2.read_text(encoding="utf-8", errors="ignore")
    def grab(pat):
        m = re.search(pat, txt); return float(m.group(1)) if m else float("nan")
    rows.append({
        "branch": br,
        "auc_mean": grab(r"AUC score:\s*([0-9.]+)"),
        "auc_std":  grab(r"AUC std:\s*([0-9.]+)"),
        "pr_mean":  grab(r"PR score:\s*([0-9.]+)"),
        "pr_std":   grab(r"PR std:\s*([0-9.]+)"),
        "train_s":  grab(r"Training time:\s*([0-9.]+)"),
        "test_s":   grab(r"Testing time:\s*([0-9.]+)")
    })

df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
display(df)

# Save CSV + LaTeX for appendix
csv_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.csv"
df.to_csv(csv_path, index=False)

latex = [
    "\\begin{table}[h]",
    "\\centering",
    "\\begin{tabular}{rcccc}",
    "\\toprule",
    "Branch & ROC-AUC (mean$\\pm$std) & PR-AUC (mean$\\pm$std) & Train (s) & Test (s) \\\\",
    "\\midrule",
]
for _, r in df.iterrows():
    latex.append(f"{int(r.branch)} & {r.auc_mean:.4f}$\\pm${r.auc_std:.4f} & {r.pr_mean:.4f}$\\pm${r.pr_std:.4f} & {r.train_s:.2f} & {r.test_s:.2f} \\\\")
latex += ["\\bottomrule","\\end{tabular}",f"\\caption{{OptIForest ablation on \\texttt{{{DATASET}}}, threshold {THRESHOLD}.}}","\\label{tab:optiforest-breastw}","\\end{table}"]
tex_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.tex"
tex_path.write_text("\n".join(latex), encoding="utf-8")

csv_path, tex_path

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw --threshold 200 --branch 0
Dataset: breastw

L2OPT:
	AUC score:	 0.9879389850101147
	AUC std:	 0.0012605723462131878
	PR score:	 0.974630024491711
	PR std:	 0.003129673435246897
	Training time:	 25.534986877441405
	Testing time:	 24.303333266576132
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw --threshold 200 --branch 2
Dataset: breastw

L2OPT:
	AUC score:	 0.9863105783607876
	AUC std:	 0.0014345368785618237
	PR score:	 0.970486811892602
	PR std:	 0.0038244210217037965
	Training time:	 17.874875370661417
	Testing time:	 25.301837142308553
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw --threshold 200 --branch 3
Dataset: breastw

L2OPT:
	AUC score:	 0.98864638

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.987939,0.001261,0.974630,0.003130,25.534987,24.303333
1,2,0.986311,0.001435,0.970487,0.003824,17.874875,25.301837
2,3,0.988646,0.000971,0.976196,0.002539,27.732343,24.082871
3,4,0.988673,0.001028,0.976226,0.002456,27.437354,24.225027


(WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_breastw_thr200.csv'),
 WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_breastw_thr200.tex'))

## Habermans

## Conversion

In [7]:
import pathlib, zipfile, pandas as pd, numpy as np, time

REPO = pathlib.Path.cwd()
DATA = REPO / "data"; DATA.mkdir(exist_ok=True)

# 1) If the ZIP is in your downloads, move or copy it into data/ first (or set ZIP_PATH accordingly)
ZIP_PATH = DATA / "haberman+s+survival.zip"
if ZIP_PATH.exists():
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(DATA)
        print("[unzip] extracted into", DATA)

# 2) Find the raw file (UCI naming varies: 'haberman.data' or 'haberman.data.txt', etc.)
raw = None
for name in ["haberman.data", "haberman.data.txt", "haberman.csv"]:
    p = DATA / name
    if p.exists():
        raw = p; break
assert raw is not None, "Place the UCI file (haberman.data) into data/ and rerun."

# 3) Load & convert to repo format (features..., label_last)
cols = ["age","op_year","nodes","status"]
df = pd.read_csv(raw, header=None, names=cols)
# Map label: status==2 (died ≤5y) -> 1 (anomaly), status==1 -> 0 (normal)
y = (df["status"].astype(int) == 2).astype(int)
X = df.drop(columns=["status"])

OUT = DATA / "haberman.csv"
pd.DataFrame(np.c_[X.to_numpy(), y.to_numpy()]).to_csv(OUT, index=False, header=False)
print(f"[convert] wrote {OUT} with shape {X.shape[0]} x {X.shape[1]+1}")

# 4) Dataset stats (good for appendix)
rows, feats = X.shape
pos_rate = float(y.mean())
stats = f"dataset=haberman\nrows={rows}\nfeatures={feats}\npositive_rate(died<=5y as anomaly)={pos_rate:.4f}\n"
print(stats)

ART = REPO / "replication_artifacts"; ART.mkdir(exist_ok=True)
ts = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
(ART / f"dataset_haberman_stats_{ts}.txt").write_text(stats, encoding="utf-8")

[convert] wrote C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\data\haberman.csv with shape 306 x 4
dataset=haberman
rows=306
features=3
positive_rate(died<=5y as anomaly)=0.2647



79

## Running OptlForest

In [ ]:
import os, sys, time, subprocess, pathlib

REPO = pathlib.Path.cwd()
ART  = REPO / "replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD, BRANCH = "haberman", "200", "3"  # fixed depth, branch=3 as your default
ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{BRANCH}_{ts}.txt"

cmd = [sys.executable, "-u", str(REPO/"demo.py"),
       "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", BRANCH]

print("COMMAND:", " ".join(cmd))
print("LOG    ->", log)

with open(log, "w", encoding="utf-8") as f:
    p = subprocess.Popen(cmd, cwd=str(REPO),
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        print(line, end=""); f.write(line)
    rc = p.wait()
print("Exit code:", rc, "| Log:", log)

## Ablation Loop

In [9]:
import re, pandas as pd, time, subprocess, os, sys, pathlib

REPO = pathlib.Path.cwd()
ART  = REPO / "replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD = "haberman", "200"
BRANCHES = [0,2,3,4]

rows = []
for br in BRANCHES:
    ts2  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log2 = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts2}.txt"
    cmd2 = [sys.executable, "-u", str(REPO/"demo.py"),
            "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", str(br)]
    print("RUN:", " ".join(cmd2), "->", log2.name)
    with open(log2, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd2, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            print(line, end=""); f.write(line)
        p.wait()
    txt = log2.read_text(encoding="utf-8", errors="ignore")
    def grab(pat):
        m = re.search(pat, txt)
        return float(m.group(1)) if m else float("nan")
    rows.append({
        "branch": br,
        "auc_mean": grab(r"AUC score:\s*([0-9.]+)"),
        "auc_std":  grab(r"AUC std:\s*([0-9.]+)"),
        "pr_mean":  grab(r"PR score:\s*([0-9.]+)"),
        "pr_std":   grab(r"PR std:\s*([0-9.]+)"),
        "train_s":  grab(r"Training time:\s*([0-9.]+)"),
        "test_s":   grab(r"Testing time:\s*([0-9.]+)")
    })

df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
display(df)

csv_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.csv"
df.to_csv(csv_path, index=False)

latex = ["\\begin{table}[h]","\\centering","\\begin{tabular}{rcccc}","\\toprule",
         "Branch & ROC-AUC (mean$\\pm$std) & PR-AUC (mean$\\pm$std) & Train (s) & Test (s) \\\\",
         "\\midrule"]
for _, r in df.iterrows():
    latex.append(f"{int(r.branch)} & {r.auc_mean:.4f}$\\pm${r.auc_std:.4f} & "
                 f"{r.pr_mean:.4f}$\\pm${r.pr_std:.4f} & {r.train_s:.2f} & {r.test_s:.2f} \\\\")
latex += ["\\bottomrule","\\end{tabular}",
          f"\\caption{{OptIForest ablation on \\texttt{{{DATASET}}} with threshold {THRESHOLD}.}}",
          f"\\label{{tab:optiforest-{DATASET}}}","\\end{table}"]
(REPO / "replication_artifacts" / f"summary_{DATASET}_thr{THRESHOLD}.tex").write_text("\n".join(latex), encoding="utf-8")

csv_path

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset haberman --threshold 200 --branch 0 -> run_haberman_thr200_br0_20250826T150631Z.txt
Dataset: haberman

L2OPT:
	AUC score:	 0.6544307270233196
	AUC std:	 0.004326200033676641
	PR score:	 0.41417821675419014
	PR std:	 0.0076028660812538324
	Training time:	 31.308301051457722
	Testing time:	 11.870658334096273
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset haberman --threshold 200 --branch 2 -> run_haberman_thr200_br2_20250826T151720Z.txt
Dataset: haberman

L2OPT:
	AUC score:	 0.6526456332876086
	AUC std:	 0.005728733753471977
	PR score:	 0.4103304432054753
	PR std:	 0.006789923429643974
	Training time:	 14.880829524993896
	Testing time:	 12.36030740737915
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\d

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.654431,0.004326,0.414178,0.007603,31.308301,11.870658
1,2,0.652646,0.005729,0.410330,0.006790,14.880830,12.360307
2,3,0.657145,0.004681,0.416356,0.006644,38.371129,11.961454
3,4,0.655550,0.005599,0.417133,0.006676,36.994368,11.580895


WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_haberman_thr200.csv')

## Glass Detection

## Conversion

In [10]:
import pandas as pd, numpy as np, pathlib, time

REPO = pathlib.Path.cwd()
DATA = REPO / "data"; DATA.mkdir(exist_ok=True)
ART  = REPO / "replication_artifacts"; ART.mkdir(exist_ok=True)

raw = DATA / "glass.data"  # <-- place your UCI file here
assert raw.exists(), "Put UCI glass.data into data/ and rerun."

df = pd.read_csv(raw, header=None)
ncols = df.shape[1]

# UCI variants:
# - 11 columns: [ID, RI, Na, Mg, Al, Si, K, Ca, Ba, Fe, class]
# - 10 columns: [RI, Na, Mg, Al, Si, K, Ca, Ba, Fe, class]
if ncols == 11:
    features = df.iloc[:, 1:-1]           # drop ID (col 0), drop class (last)
    cls = df.iloc[:, -1].astype(int)
elif ncols == 10:
    features = df.iloc[:, :-1]            # all but last
    cls = df.iloc[:, -1].astype(int)
else:
    raise ValueError(f"Unexpected column count {ncols}; expected 10 or 11.")

# Anomaly mapping: window glass = normal (classes 1–4), non-window = anomaly (5–7)
label = cls.isin([5,6,7]).astype(int)

# Save to repo format: features..., label_last (no header)
OUT = DATA / "glass.csv"
pd.DataFrame(np.c_[features.to_numpy(), label.to_numpy()]).to_csv(OUT, index=False, header=False)
print(f"[convert] wrote {OUT} shape={features.shape[0]} x {features.shape[1]+1}")

# Stats (for appendix)
rows, feats = features.shape
pos_rate = float(label.mean())
stats = f"dataset=glass\nrows={rows}\nfeatures={feats}\npositive_rate(non-window as anomaly)={pos_rate:.4f}\n"
print(stats)

ts = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
(ART / f"dataset_glass_stats_{ts}.txt").write_text(stats, encoding="utf-8")

[convert] wrote C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\data\glass.csv shape=214 x 10
dataset=glass
rows=214
features=9
positive_rate(non-window as anomaly)=0.2383



78

## Running 

In [11]:
import os, sys, time, subprocess, pathlib

REPO = pathlib.Path.cwd()
ART  = REPO / "replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD, BRANCH = "glass", "200", "3"  # fixed depth, branch=3
ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{BRANCH}_{ts}.txt"

cmd = [sys.executable, "-u", str(REPO/"demo.py"),
       "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", BRANCH]

print("COMMAND:", " ".join(cmd))
print("LOG    ->", log)

with open(log, "w", encoding="utf-8") as f:
    p = subprocess.Popen(cmd, cwd=str(REPO),
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        print(line, end=""); f.write(line)
    rc = p.wait()
print("Exit code:", rc, "| Log:", log)

COMMAND: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset glass --threshold 200 --branch 3
LOG    -> C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_glass_thr200_br3_20250826T155155Z.txt
Dataset: glass

L2OPT:
	AUC score:	 0.9057059224507799
	AUC std:	 0.005518639774384812
	PR score:	 0.6328373177667619
	PR std:	 0.013013866459354257
	Training time:	 7.424543142318726
	Testing time:	 5.703059800465902
Exit code: 0 | Log: C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_glass_thr200_br3_20250826T155155Z.txt


## Ablation Loop

In [12]:
import re, pandas as pd, time, subprocess, os, sys, pathlib

REPO = pathlib.Path.cwd()
ART  = REPO / "replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD = "glass", "200"
BRANCHES = [0,2,3,4]

rows = []
for br in BRANCHES:
    ts2  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log2 = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts2}.txt"
    cmd2 = [sys.executable, "-u", str(REPO/"demo.py"),
            "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", str(br)]
    print("RUN:", " ".join(cmd2), "->", log2.name)
    with open(log2, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd2, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            print(line, end=""); f.write(line)
        p.wait()
    txt = log2.read_text(encoding="utf-8", errors="ignore")
    def grab(pat):
        m = re.search(pat, txt)
        return float(m.group(1)) if m else float("nan")
    rows.append({
        "branch": br,
        "auc_mean": grab(r"AUC score:\s*([0-9.]+)"),
        "auc_std":  grab(r"AUC std:\s*([0-9.]+)"),
        "pr_mean":  grab(r"PR score:\s*([0-9.]+)"),
        "pr_std":   grab(r"PR std:\s*([0-9.]+)"),
        "train_s":  grab(r"Training time:\s*([0-9.]+)"),
        "test_s":   grab(r"Testing time:\s*([0-9.]+)")
    })

df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
display(df)

csv_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.csv"
df.to_csv(csv_path, index=False)

latex = ["\\begin{table}[h]","\\centering","\\begin{tabular}{rcccc}","\\toprule",
         "Branch & ROC-AUC (mean$\\pm$std) & PR-AUC (mean$\\pm$std) & Train (s) & Test (s) \\\\",
         "\\midrule"]
for _, r in df.iterrows():
    latex.append(f"{int(r.branch)} & {r.auc_mean:.4f}$\\pm${r.auc_std:.4f} & "
                 f"{r.pr_mean:.4f}$\\pm${r.pr_std:.4f} & {r.train_s:.2f} & {r.test_s:.2f} \\\\")
latex += ["\\bottomrule","\\end{tabular}",
          f"\\caption{{OptIForest ablation on \\texttt{{{DATASET}}} with threshold {THRESHOLD}.}}",
          f"\\label{{tab:optiforest-{DATASET}}}","\\end{table}"]
(REPO / "replication_artifacts" / f"summary_{DATASET}_thr{THRESHOLD}.tex").write_text("\n".join(latex), encoding="utf-8")

csv_path

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset glass --threshold 200 --branch 0 -> run_glass_thr200_br0_20250826T155805Z.txt
Dataset: glass

L2OPT:
	AUC score:	 0.907646657845142
	AUC std:	 0.004523946138691698
	PR score:	 0.6385189914209423
	PR std:	 0.009071826622970857
	Training time:	 7.511750729878743
	Testing time:	 5.784227275848389
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset glass --threshold 200 --branch 2 -> run_glass_thr200_br2_20250826T160125Z.txt
Dataset: glass

L2OPT:
	AUC score:	 0.9074140903805284
	AUC std:	 0.0034702588754029927
	PR score:	 0.6384347384407613
	PR std:	 0.015526494454281168
	Training time:	 7.089921728769938
	Testing time:	 5.774336163202921
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset glass 

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.907647,0.004524,0.638519,0.009072,7.511751,5.784227
1,2,0.907414,0.003470,0.638435,0.015526,7.089922,5.774336
2,3,0.907141,0.006509,0.637875,0.014520,7.519284,5.671220
3,4,0.910871,0.002901,0.650244,0.012704,7.462996,5.849592


WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_glass_thr200.csv')

## Parkinsons

## Conversion

In [2]:
import pandas as pd, numpy as np, pathlib, time

REPO = pathlib.Path.cwd()
DATA = REPO/"data"; DATA.mkdir(exist_ok=True)
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)

raw = DATA/"parkinsons.data"     # put the UCI file here
assert raw.exists(), "Place parkinsons.data into data/ and rerun."

# UCI 'parkinsons.data' usually has a header row; 'status' is the target, 'name' is an ID-like column
df = pd.read_csv(raw)

# If the file came without headers, fallback (rare)
if "status" not in df.columns:
    df = pd.read_csv(raw, header=None)
    # last column should be 'status'; first often 'name'
    df.columns = [*(f"f{i}" for i in range(df.shape[1]-2)), "status_tmp", "status"]
    # drop the penultimate 'status_tmp' if the layout mismatched; keep the final 'status'
    if "status_tmp" in df.columns:
        df = df.drop(columns=["status_tmp"])

# Build features/label
label = df["status"].astype(int)                     # 1 = Parkinson's, 0 = healthy
features = df.drop(columns=["status","name"], errors="ignore")

# Save repo format: features..., label_last (no header)
OUT = DATA/"parkinsons.csv"
pd.DataFrame(np.c_[features.to_numpy(), label.to_numpy()]).to_csv(OUT, index=False, header=False)
print(f"[convert] wrote {OUT}  shape={features.shape[0]} x {features.shape[1]+1}")

# Stats for appendix
rows, feats = features.shape
pos_rate = float(label.mean())
stats = f"dataset=parkinsons\nrows={rows}\nfeatures={feats}\npositive_rate(status==1 as anomaly)={pos_rate:.4f}\n"
print(stats)

ts = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
(ART/f"dataset_parkinsons_stats_{ts}.txt").write_text(stats, encoding="utf-8")

[convert] wrote C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\data\parkinsons.csv  shape=195 x 23
dataset=parkinsons
rows=195
features=22
positive_rate(status==1 as anomaly)=0.7538



83

## Running

In [3]:
import os, sys, time, subprocess, pathlib

REPO = pathlib.Path.cwd()
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD, BRANCH = "parkinsons", "200", "3"  # same defaults you’ve been using
ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{BRANCH}_{ts}.txt"

cmd = [sys.executable, "-u", str(REPO/"demo.py"),
       "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", BRANCH]

print("COMMAND:", " ".join(cmd))
print("LOG    ->", log)

with open(log, "w", encoding="utf-8") as f:
    p = subprocess.Popen(cmd, cwd=str(REPO),
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        print(line, end=""); f.write(line)
    rc = p.wait()
print("Exit code:", rc, "| Log:", log)

COMMAND: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset parkinsons --threshold 200 --branch 3
LOG    -> C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_parkinsons_thr200_br3_20250826T180234Z.txt
Dataset: parkinsons

L2OPT:
	AUC score:	 0.40611300075585793
	AUC std:	 0.01090501033120998
	PR score:	 0.7014902428687138
	PR std:	 0.006182108326006693
	Training time:	 3.5115992546081545
	Testing time:	 1.017062250773112
Exit code: 0 | Log: C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\run_parkinsons_thr200_br3_20250826T180234Z.txt


## Ablation Loop

In [4]:
import re, pandas as pd, time, subprocess, os, sys, pathlib

REPO = pathlib.Path.cwd()
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD = "parkinsons", "200"
BRANCHES = [0,2,3,4]

rows = []
for br in BRANCHES:
    ts2  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log2 = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts2}.txt"
    cmd2 = [sys.executable, "-u", str(REPO/"demo.py"),
            "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", str(br)]
    print("RUN:", " ".join(cmd2), "->", log2.name)
    with open(log2, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd2, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            print(line, end=""); f.write(line)
        p.wait()
    txt = log2.read_text(encoding="utf-8", errors="ignore")
    def grab(pat):
        m = re.search(pat, txt); return float(m.group(1)) if m else float("nan")
    rows.append({
        "branch": br,
        "auc_mean": grab(r"AUC score:\s*([0-9.]+)"),
        "auc_std":  grab(r"AUC std:\s*([0-9.]+)"),
        "pr_mean":  grab(r"PR score:\s*([0-9.]+)"),
        "pr_std":   grab(r"PR std:\s*([0-9.]+)"),
        "train_s":  grab(r"Training time:\s*([0-9.]+)"),
        "test_s":   grab(r"Testing time:\s*([0-9.]+)")
    })

df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
display(df)

csv_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.csv"
df.to_csv(csv_path, index=False)

latex = ["\\begin{table}[h]","\\centering","\\begin{tabular}{rcccc}","\\toprule",
         "Branch & ROC-AUC (mean$\\pm$std) & PR-AUC (mean$\\pm$std) & Train (s) & Test (s) \\\\",
         "\\midrule"]
for _, r in df.iterrows():
    latex.append(f"{int(r.branch)} & {r.auc_mean:.4f}$\\pm${r.auc_std:.4f} & "
                 f"{r.pr_mean:.4f}$\\pm${r.pr_std:.4f} & {r.train_s:.2f} & {r.test_s:.2f} \\\\")
latex += ["\\bottomrule","\\end{tabular}",
          f"\\caption{{OptIForest ablation on \\texttt{{{DATASET}}} with threshold {THRESHOLD}.}}",
          f"\\label{{tab:optiforest-{DATASET}}}","\\end{table}"]
(REPO/"replication_artifacts"/f"summary_{DATASET}_thr{THRESHOLD}.tex").write_text("\n".join(latex), encoding="utf-8")

csv_path

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset parkinsons --threshold 200 --branch 0 -> run_parkinsons_thr200_br0_20250826T180343Z.txt
Dataset: parkinsons

L2OPT:
	AUC score:	 0.40475245653817077
	AUC std:	 0.009539496777455764
	PR score:	 0.7018451404189161
	PR std:	 0.0042153133506582134
	Training time:	 3.5236051082611084
	Testing time:	 1.0183056672414144
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset parkinsons --threshold 200 --branch 2 -> run_parkinsons_thr200_br2_20250826T180452Z.txt
Dataset: parkinsons

L2OPT:
	AUC score:	 0.4041477702191988
	AUC std:	 0.007312278170054215
	PR score:	 0.7015340442790191
	PR std:	 0.004871454269601618
	Training time:	 3.557615041732788
	Testing time:	 0.9809576511383057
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIF

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.404752,0.009539,0.701845,0.004215,3.523605,1.018306
1,2,0.404148,0.007312,0.701534,0.004871,3.557615,0.980958
2,3,0.408314,0.007856,0.703007,0.003297,3.471760,1.010175
3,4,0.408910,0.008084,0.703510,0.004376,3.531930,1.006675


WindowsPath('C:/Users/yonda/Downloads/OptIForest-main/OptIForest-main/replication_artifacts/summary_parkinsons_thr200.csv')

## BankNote Authorization

## Conversion

In [1]:
import pandas as pd, numpy as np, pathlib, time

REPO = pathlib.Path.cwd()
DATA = REPO/"data"; DATA.mkdir(exist_ok=True)
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)

# 1) Put the UCI file into data/ with this exact name:
src = DATA/"data_banknote_authentication.txt"
assert src.exists(), "Place data_banknote_authentication.txt into the repo's data/ folder and re-run."

# UCI banknote: 4 numeric features + 1 label (0=authentic, 1=fake), comma-separated, no header
df = pd.read_csv(src, header=None)
assert df.shape[1] == 5, f"Expected 5 columns (4 feats + label). Got {df.shape[1]}."

X = df.iloc[:, :4].to_numpy()
y = df.iloc[:, 4].astype(int).to_numpy()   # treat class==1 (fake) as anomaly

out_csv = DATA/"banknote.csv"
pd.DataFrame(np.c_[X, y]).to_csv(out_csv, index=False, header=False)
print(f"[convert] wrote {out_csv}  shape={X.shape[0]} x {X.shape[1]+1}")

# Save small dataset facts (for appendix)
pos_rate = float(y.mean())
stats = f"dataset=banknote\nrows={X.shape[0]}\nfeatures={X.shape[1]}\npositive_rate(fake_as_anomaly)={pos_rate:.4f}\n"
print(stats)

ts = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
(ART/f"dataset_banknote_stats_{ts}.txt").write_text(stats, encoding="utf-8")

[convert] wrote C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\data\banknote.csv  shape=1372 x 5
dataset=banknote
rows=1372
features=4
positive_rate(fake_as_anomaly)=0.4446



76

## Running and branch ablation

In [5]:
import os, sys, time, subprocess, pathlib, re, pandas as pd

REPO = pathlib.Path.cwd()
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET   = "banknote"   # because we saved data/banknote.csv
THRESHOLD = "200"        # keep consistent with your other datasets
BRANCHES  = [0,2,3,4]

rows = []
for br in BRANCHES:
    ts = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts}.txt"
    cmd = [sys.executable, "-u", str(REPO/"demo.py"),
           "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", str(br)]
    print("RUN:", " ".join(cmd), "->", log.name)

    with open(log, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            print(line, end=""); f.write(line)
        p.wait()

    # Parse metrics from the log
    txt = log.read_text(encoding="utf-8", errors="ignore")
    def grab(pat):
        m = re.search(pat, txt)
        return float(m.group(1)) if m else float("nan")

    rows.append({
        "branch": br,
        "auc_mean": grab(r"AUC score:\s*([0-9.]+)"),
        "auc_std":  grab(r"AUC std:\s*([0-9.]+)"),
        "pr_mean":  grab(r"PR score:\s*([0-9.]+)"),
        "pr_std":   grab(r"PR std:\s*([0-9.]+)"),
        "train_s":  grab(r"Training time:\s*([0-9.]+)"),
        "test_s":   grab(r"Testing time:\s*([0-9.]+)")
    })

df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
display(df)

# Save CSV and LaTeX table snippet for your ACL proposal
csv_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.csv"
df.to_csv(csv_path, index=False)

latex = ["\\begin{table}[h]","\\centering","\\begin{tabular}{rcccc}","\\toprule",
         "Branch & ROC-AUC (mean$\\pm$std) & PR-AUC (mean$\\pm$std) & Train (s) & Test (s) \\\\",
         "\\midrule"]
for _, r in df.iterrows():
    latex.append(f"{int(r.branch)} & {r.auc_mean:.4f}$\\pm${r.auc_std:.4f} & "
                 f"{r.pr_mean:.4f}$\\pm${r.pr_std:.4f} & {r.train_s:.2f} & {r.test_s:.2f} \\\\")
latex += ["\\bottomrule","\\end{tabular}",
          f"\\caption{{OptIForest ablation on \\texttt{{{DATASET}}} (threshold {THRESHOLD}).}}",
          f"\\label{{tab:optiforest-{DATASET}}}","\\end{table}"]

tex_path = ART / f"summary_{DATASET}_thr{THRESHOLD}.tex"
tex_path.write_text("\n".join(latex), encoding="utf-8")

print("Saved:", csv_path.name, "and", tex_path.name, "in", ART)

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset banknote --threshold 200 --branch 0 -> run_banknote_thr200_br0_20250826T180818Z.txt
Dataset: banknote

L2OPT:
	AUC score:	 0.4842998723520216
	AUC std:	 0.024550654848220395
	PR score:	 0.4377209800546434
	PR std:	 0.017631960145259504
	Training time:	 22.341855748494467
	Testing time:	 47.68959253629048
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset banknote --threshold 200 --branch 2 -> run_banknote_thr200_br2_20250826T182550Z.txt
Dataset: banknote

L2OPT:
	AUC score:	 0.48935114668043556
	AUC std:	 0.026736518732754917
	PR score:	 0.44541469636161307
	PR std:	 0.018043622330863596
	Training time:	 19.666068935394286
	Testing time:	 49.7356475353241
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\dem

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.484300,0.024551,0.437721,0.017632,22.341856,47.689593
1,2,0.489351,0.026737,0.445415,0.018044,19.666069,49.735648
2,3,0.492877,0.022250,0.446477,0.019592,23.454459,47.628494
3,4,0.494659,0.016277,0.441826,0.010877,23.148546,47.038027


Saved: summary_banknote_thr200.csv and summary_banknote_thr200.tex in C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts


## For New Data using BreastW from the Breast Cancer Wisconsin dataset

In [2]:
# Creates:
#   data/breastw_points_1pct.csv,   data/breastw_points_5pct.csv
#   data/breastw_cluster_1pct.csv,  data/breastw_cluster_5pct.csv
#   data/breastw_reinject_1pct.csv, data/breastw_reinject_5pct.csv
# And logs generation stats/params into replication_artifacts/.

import numpy as np, pandas as pd, pathlib, time, json
from sklearn.preprocessing import StandardScaler

REPO = pathlib.Path.cwd()
DATA = REPO/"data"; DATA.mkdir(exist_ok=True)
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)

BASE = "breastw"
SRC  = DATA/f"{BASE}.csv"
assert SRC.exists(), f"Expected {SRC} (features..., label_last)."

df = pd.read_csv(SRC, header=None)
X = df.iloc[:, :-1].to_numpy(float)
y = df.iloc[:,  -1].astype(int).to_numpy()

# normals = benign (0), anomalies = malignant (1)
Xn = X[y==0]
n, d = Xn.shape

rng = np.random.default_rng(42)
sc  = StandardScaler().fit(Xn)
Zn  = sc.transform(Xn)

def _write(name, Xmat, yvec, params):
    out_csv = DATA/f"{name}.csv"
    pd.DataFrame(np.c_[Xmat, yvec]).to_csv(out_csv, index=False, header=False)
    rows, feats = Xmat.shape
    pos = float(np.mean(yvec))
    ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    (ART/f"dataset_{name}_stats_{ts}.txt").write_text(
        f"dataset={name}\nrows={rows}\nfeatures={feats}\npositive_rate={pos:.4f}\n", encoding="utf-8")
    (ART/f"generation_{name}_{ts}.json").write_text(json.dumps(params, indent=2), encoding="utf-8")
    print(f"[wrote] {out_csv.name}  shape={rows}x{feats+1}  pos_rate={pos:.4f}")

def make_point(cont=0.01, spike_sigma=6.0):
    k = max(1, int(round(cont * n)))
    idx = rng.choice(n, size=k, replace=False)
    Zp  = Zn[idx].copy()
    for i in range(k):
        cols = rng.choice(d, size=rng.integers(1,3), replace=False)
        Zp[i, cols] += rng.normal(0, 1, size=len(cols)) * spike_sigma
    Xp = sc.inverse_transform(Zp)
    Xnew = np.vstack([Xn, Xp])
    ynew = np.concatenate([np.zeros(len(Xn), int), np.ones(k, int)])
    return Xnew, ynew, {"type":"point","contamination":cont,"spike_sigma":spike_sigma,"seed":42}

def make_cluster(cont=0.01, shift_sigma=6.0, cluster_sigma=0.5):
    k = max(1, int(round(cont * n)))
    dirv = rng.normal(size=d); dirv /= (np.linalg.norm(dirv)+1e-9)
    Zc_center = Zn.mean(axis=0) + shift_sigma*dirv
    Zc = Zc_center + rng.normal(scale=cluster_sigma, size=(k, d))
    Xc = sc.inverse_transform(Zc)
    Xnew = np.vstack([Xn, Xc])
    ynew = np.concatenate([np.zeros(len(Xn), int), np.ones(k, int)])
    return Xnew, ynew, {"type":"cluster","contamination":cont,
                        "shift_sigma":shift_sigma,"cluster_sigma":cluster_sigma,"seed":42}

def make_reinject(cont=0.01):
    Xpos = X[y==1]
    assert len(Xpos)>0, "No malignant examples available to reinject."
    n_norm = len(Xn)
    target_total = int(round(n_norm/(1.0-cont)))
    k = max(1, target_total - n_norm)
    k = min(k, len(Xpos))
    pos_idx = rng.choice(len(Xpos), size=k, replace=False)
    Xnew = np.vstack([Xn, Xpos[pos_idx]])
    ynew = np.concatenate([np.zeros(n_norm, int), np.ones(k, int)])
    return Xnew, ynew, {"type":"reinject","contamination":cont,"seed":42,"source":"real_malignant"}

for cont in (0.01, 0.05):
    Xp, yp, pp = make_point(cont=cont);     _write(f"{BASE}_points_{int(cont*100)}pct",  Xp, yp, pp)
    Xc, yc, pc = make_cluster(cont=cont);   _write(f"{BASE}_cluster_{int(cont*100)}pct", Xc, yc, pc)
    Xr, yr, pr = make_reinject(cont=cont);  _write(f"{BASE}_reinject_{int(cont*100)}pct", Xr, yr, pr)

[wrote] breastw_points_1pct.csv  shape=448x10  pos_rate=0.0089
[wrote] breastw_cluster_1pct.csv  shape=448x10  pos_rate=0.0089
[wrote] breastw_reinject_1pct.csv  shape=448x10  pos_rate=0.0089
[wrote] breastw_points_5pct.csv  shape=466x10  pos_rate=0.0472
[wrote] breastw_cluster_5pct.csv  shape=466x10  pos_rate=0.0472
[wrote] breastw_reinject_5pct.csv  shape=467x10  pos_rate=0.0493


## Running

In [3]:
# Runs each created dataset once (branch=3, threshold=200) and saves a log.
import os, sys, time, subprocess, pathlib

REPO = pathlib.Path.cwd()
DATA = REPO/"data"
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

CREATED = [
    "breastw_points_1pct","breastw_cluster_1pct","breastw_reinject_1pct",
    "breastw_points_5pct","breastw_cluster_5pct","breastw_reinject_5pct"
]
CREATED = [ds for ds in CREATED if (DATA/f"{ds}.csv").exists()]

for ds in CREATED:
    ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log = ART / f"run_{ds}_thr200_br3_{ts}.txt"
    cmd = [sys.executable, "-u", str(REPO/"demo.py"),
           "--dataset", ds, "--threshold", "200", "--branch", "3"]
    print("RUN:", " ".join(cmd), "->", log.name)
    with open(log, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout: print(line, end=""); f.write(line)
        p.wait()
print("\n[done] logs saved in", ART)

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw_points_1pct --threshold 200 --branch 3 -> run_breastw_points_1pct_thr200_br3_20250827T044016Z.txt
Dataset: breastw_points_1pct

L2OPT:
	AUC score:	 0.9058933933933934
	AUC std:	 0.00550868958794118
	PR score:	 0.07482555381142918
	PR std:	 0.004329500550997652
	Training time:	 17.942912181218464
	Testing time:	 14.142622248331707
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw_cluster_1pct --threshold 200 --branch 3 -> run_breastw_cluster_1pct_thr200_br3_20250827T044819Z.txt
Dataset: breastw_cluster_1pct

L2OPT:
	AUC score:	 0.9144144144144145
	AUC std:	 0.007896277765751733
	PR score:	 0.06016493383301461
	PR std:	 0.005332047067799071
	Training time:	 19.639004230499268
	Testing time:	 15.554672320683798
RUN: C:\Users\yonda\.conda\envs\optiforest38\pyt

## Ablation Loop

In [1]:
# Ablate {0,2,4} on breastw_cluster_1pct (change DATASET if you prefer)
import re, pandas as pd, os, sys, time, subprocess, pathlib
REPO = pathlib.Path.cwd(); ART = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

DATASET, THRESHOLD, BRANCHES = "breastw_cluster_1pct", "200", [0,2,4]
rows = []
for br in BRANCHES:
    ts  = time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())
    log = ART / f"run_{DATASET}_thr{THRESHOLD}_br{br}_{ts}.txt"
    cmd = [sys.executable, "-u", str(REPO/"demo.py"),
           "--dataset", DATASET, "--threshold", THRESHOLD, "--branch", str(br)]
    print("RUN:", " ".join(cmd), "->", log.name)
    with open(log, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout: print(line, end=""); f.write(line)
        p.wait()
    txt = log.read_text(encoding="utf-8", errors="ignore")
    g = lambda pat: float(re.search(pat, txt).group(1)) if re.search(pat, txt) else float("nan")
    rows.append({"branch":br,"auc_mean":g(r"AUC score:\s*([0-9.]+)"),"auc_std":g(r"AUC std:\s*([0-9.]+)"),
                 "pr_mean":g(r"PR score:\s*([0-9.]+)"),"pr_std":g(r"PR std:\s*([0-9.]+)"),
                 "train_s":g(r"Training time:\s*([0-9.]+)"),"test_s":g(r"Testing time:\s*([0-9.]+)")})
df = pd.DataFrame(rows).sort_values("branch").reset_index(drop=True)
display(df)
df.to_csv(ART/f"summary_{DATASET}_thr{THRESHOLD}.csv", index=False)

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw_cluster_1pct --threshold 200 --branch 0 -> run_breastw_cluster_1pct_thr200_br0_20250828T014410Z.txt
Dataset: breastw_cluster_1pct

L2OPT:
	AUC score:	 0.9128003003003002
	AUC std:	 0.006455583431443135
	PR score:	 0.0588565843656416
	PR std:	 0.004233643279459944
	Training time:	 18.285171906153362
	Testing time:	 14.617285060882569
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset breastw_cluster_1pct --threshold 200 --branch 2 -> run_breastw_cluster_1pct_thr200_br2_20250828T015228Z.txt
Dataset: breastw_cluster_1pct

L2OPT:
	AUC score:	 0.9108108108108108
	AUC std:	 0.010501390294481747
	PR score:	 0.058410154993773604
	PR std:	 0.005919806374766337
	Training time:	 15.294818655649822
	Testing time:	 15.420495144526164
RUN: C:\Users\yonda\.conda\envs\optiforest38

,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,0,0.912800,0.006456,0.058857,0.004234,18.285172,14.617285
1,2,0.910811,0.010501,0.058410,0.005920,15.294819,15.420495
2,4,0.907845,0.010547,0.056337,0.006174,17.616893,14.217918


# AD DATASET

In [2]:

import os, sys, re, time, subprocess
from pathlib import Path
import pandas as pd

# --- paths ---
REPO = Path.cwd()
DEMO = REPO / "demo.py"
DATA = REPO / "data"
ART  = REPO / "replication_artifacts"
ART.mkdir(exist_ok=True)

assert DEMO.exists(), f"demo.py not found at {DEMO}"
assert DATA.exists(), f"data/ folder not found at {DATA}"
dataset = "ad"        # <- full AD
thr     = "403"
branches = [0, 2, 3, 4]
os.environ["PYTHONPATH"] = str(REPO)

def _ts():
    return time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())

def run_and_log(branch: int):
    """Run one branch, tee stdout to a log file, and parse metrics."""
    log = ART / f"run_{dataset}_thr{thr}_br{branch}_{_ts()}.txt"
    cmd = [sys.executable, "-u", str(DEMO),
           "--dataset", dataset, "--threshold", thr, "--branch", str(branch)]
    print("RUN:", " ".join(cmd))
    print("LOG:", log.name)

    
    with open(log, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd, cwd=str(REPO),
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                             text=True, bufsize=1)
        out_lines = []
        for line in p.stdout:
            print(line, end="")
            f.write(line)
            out_lines.append(line)
        p.wait()

    text = "".join(out_lines)

    # parse metrics 
    def grab(name):
        m = re.search(rf"{name}\s*:\s*([0-9]*\.?[0-9]+)", text)
        return float(m.group(1)) if m else float("nan")

    row = dict(
        dataset=dataset,
        branch=branch,
        auc_mean = grab(r"AUC score"),
        auc_std  = grab(r"AUC std"),
        pr_mean  = grab(r"PR score"),
        pr_std   = grab(r"PR std"),
        train_s  = grab(r"Training time"),
        test_s   = grab(r"Testing time"),
        log_file = log.name,
    )
    return row

env_log = ART / f"env_thr{thr}_{_ts()}.txt"
with open(env_log, "w", encoding="utf-8") as f:
    f.write(subprocess.run([sys.executable, "-V"], capture_output=True, text=True).stdout)
    f.write(subprocess.run([sys.executable, "-m", "pip", "freeze"], capture_output=True, text=True).stdout)
print("Saved env snapshot ->", env_log.name)

Saved env snapshot -> env_thr403_20250828T043125Z.txt


In [3]:
rows = []
for br in branches:
    rows.append(run_and_log(br))

df = pd.DataFrame(rows).sort_values("branch")
display(df)

out_csv = ART / "summary_ad_thr403.csv"
df.to_csv(out_csv, index=False)
print("Wrote:", out_csv)

RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad --threshold 403 --branch 0
LOG: run_ad_thr403_br0_20250828T043217Z.txt
Dataset: ad

L2OPT:
	AUC score:	 0.7727636654872087
	AUC std:	 0.005400132197132069
	PR score:	 0.4294883441597089
	PR std:	 0.040130640887967264
	Training time:	 53.76094055175781
	Testing time:	 136.47691904703777
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad --threshold 403 --branch 2
LOG: run_ad_thr403_br2_20250828T051958Z.txt
Dataset: ad

L2OPT:
	AUC score:	 0.7689623603578547
	AUC std:	 0.0054545506861708075
	PR score:	 0.39614482168706844
	PR std:	 0.04594279632127188
	Training time:	 51.79112373987834
	Testing time:	 144.1331021944682
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad --threshold 403

,dataset,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s,log_file
0,ad,0,0.772764,0.005400,0.429488,0.040131,53.760941,136.476919,run_ad_thr403_br0_20250828T043217Z.txt
1,ad,2,0.768962,0.005455,0.396145,0.045943,51.791124,144.133102,run_ad_thr403_br2_20250828T051958Z.txt
2,ad,3,0.777039,0.006097,0.451815,0.029285,51.909465,132.113353,run_ad_thr403_br3_20250828T060859Z.txt
3,ad,4,0.775741,0.005196,0.437057,0.039039,51.861022,136.182364,run_ad_thr403_br4_20250828T065508Z.txt


Wrote: C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\summary_ad_thr403.csv


In [4]:
import os, sys, re, time, subprocess
from pathlib import Path
import pandas as pd

REPO = Path.cwd()
DEMO = REPO/"demo.py"
ART  = REPO/"replication_artifacts"; ART.mkdir(exist_ok=True)
os.environ["PYTHONPATH"] = str(REPO)

dataset = "ad"          # ensure data/ad.csv is the FULL dataset
thr     = "200"
branches = [0,2,3,4]

def _ts(): return time.strftime("%Y%m%dT%H%M%SZ", time.gmtime())

def run_and_parse(branch):
    log = ART/f"run_{dataset}_thr{thr}_br{branch}_{_ts()}.txt"
    cmd = [sys.executable, "-u", str(DEMO),
           "--dataset", dataset, "--threshold", thr, "--branch", str(branch)]
    print("RUN:", " ".join(cmd), "->", log.name)
    with open(log, "w", encoding="utf-8") as f:
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                             text=True, bufsize=1, cwd=str(REPO))
        out = []
        for line in p.stdout: print(line, end=""); f.write(line); out.append(line)
        p.wait()
    txt = "".join(out)
    grab = lambda k: float(re.search(rf"{k}\s*:\s*([0-9]*\.?[0-9]+)", txt).group(1))
    return dict(dataset=dataset, branch=branch,
                auc_mean=grab("AUC score"), auc_std=grab("AUC std"),
                pr_mean=grab("PR score"),  pr_std=grab("PR std"),
                train_s=grab("Training time"), test_s=grab("Testing time"))

rows = [run_and_parse(b) for b in branches]
df = pd.DataFrame(rows).sort_values("branch")
display(df)
out_csv = ART/"summary_ad_thr200_full.csv"
df.to_csv(out_csv, index=False); print("Wrote:", out_csv)


RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad --threshold 200 --branch 0 -> run_ad_thr200_br0_20250828T151213Z.txt
Dataset: ad

L2OPT:
	AUC score:	 0.773721962123436
	AUC std:	 0.004460200475616314
	PR score:	 0.4458571959009529
	PR std:	 0.02412876296049597
	Training time:	 60.005445369084676
	Testing time:	 167.84761912027994
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad --threshold 200 --branch 2 -> run_ad_thr200_br2_20250828T160920Z.txt
Dataset: ad

L2OPT:
	AUC score:	 0.7663325813645658
	AUC std:	 0.004278327842224546
	PR score:	 0.38533367982287536
	PR std:	 0.025815602655575764
	Training time:	 56.207716433207196
	Testing time:	 177.39990139007568
RUN: C:\Users\yonda\.conda\envs\optiforest38\python.exe -u C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\demo.py --dataset ad --threshold 200 --

,dataset,branch,auc_mean,auc_std,pr_mean,pr_std,train_s,test_s
0,ad,0,0.773722,0.004460,0.445857,0.024129,60.005445,167.847619
1,ad,2,0.766333,0.004278,0.385334,0.025816,56.207716,177.399901
2,ad,3,0.775650,0.004806,0.447493,0.027804,60.470155,164.181951
3,ad,4,0.774773,0.005531,0.438158,0.039374,60.497137,164.742011


Wrote: C:\Users\yonda\Downloads\OptIForest-main\OptIForest-main\replication_artifacts\summary_ad_thr200_full.csv


In [3]:
import sys
print(f"Python: {sys.version}")
print(f"Conda env: optiforest38")

import numpy, sklearn, pandas
print(f"numpy: {numpy.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"pandas: {pandas.__version__}")

Python: 3.8.20 (default, Oct  3 2024, 15:19:54) [MSC v.1929 64 bit (AMD64)]
Conda env: optiforest38
numpy: 1.20.1
scikit-learn: 0.22.1
pandas: 1.4.1


In [1]:
import pandas as pd

steam_lab = pd.read_csv("data/steam_labelled.csv", header=None)
steam_full = pd.read_csv("data/steam_full.csv", header=None)

print("steam_labelled shape:", steam_lab.shape)
print("steam_full shape:", steam_full.shape)

print("\nLabel distribution in steam_labelled (last column):")
print(steam_lab.iloc[:, -1].value_counts())

print("\nLabel distribution in steam_full (last column):")
print(steam_full.iloc[:, -1].value_counts())

print("\nHead of steam_labelled:")
print(steam_lab.head())

steam_labelled shape: (253, 59)
steam_full shape: (4500, 59)

Label distribution in steam_labelled (last column):
0    162
1     91
Name: 58, dtype: int64

Label distribution in steam_full (last column):
0    4409
1      91
Name: 58, dtype: int64

Head of steam_labelled:
          0       1   2   3         4         5   6   7   8   9   ...  49  50  \
0  208338323  553850  17   2  0.058824  0.058824   0   0   0   1  ...   1   0   
1  208259553  553850  13   2  0.923077  0.000000   0   0   0   1  ...   0   1   
2  208194185  553850  13   1  0.769231  0.230769   3   0   0   1  ...   0   1   
3  208188065  553850   4   1  0.000000  0.000000   0   0   0   1  ...   0   0   
4  208187312  553850  27   4  0.888889  0.000000   0   0   0   1  ...   0   1   

   51  52  53  54  55  56  57  58  
0   1   1   1   0   0   0   1   0  
1   1   1   1   0   1   1   1   0  
2   1   1   1   0   1   1   1   1  
3   1   1   1   0   1   1   1   0  
4   0   1   1   0   1   1   1   0  

[5 rows x 59 columns]
